# Bayesian Hierarchical Modeling with PYMC3: The Ames Housing Dataset

The purpose of this notebook is to try to put into practice some of the ideas in Richard McElrath's Statistical Rethinking. Specifically chapter *insert chapter here*, where he deals with hierarchical models. In addition I will be drawing heavily from [this blog post](https://twiecki.io/blog/2014/03/17/bayesian-glms-3/) which uses the radon dataset to showcase the key aspects of hierarchical linear regression as well as ways to visualize and evaluate the model

## The Dataset

The Ames Housing dataset can be best thought of as a modern update on the classic Boston Housing dataset. It includes quite a few more features with a mix of categorical and continuous variables with plenty of missing values.

Its a great dataset for diving into hierarchical linear regression since the level of the neighborhood is clear way in which to pool our observations.

A full list of the features and their descriptions can be found in ```data_description.txt```.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [ ]:
# Create an index for each neighborhood

neighborhood_idxs = dict(zip(ames['Neighborhood'].unique(), range(ames['Neighborhood'].nunique())))
ames['Neighborhood_code'] = ames['Neighborhood'].map(neighborhood_idxs)

In [ ]:
ames = pd.read_csv('train.csv', index_col=0)

def log_and_normalize(col):
    log_col = np.log(col)
    return (log_col - log_col.mean()) / log_col.std()

# ames[['SalePrice', 'LotArea']].apply(log_and_normalize)
ames[['log_SalePrice', 'log_LotArea']] = np.log(ames[['SalePrice', 'LotArea']])

There are 80 different features included in this dataset but as a first pass lets just stick to the basics and only look at neighborhood and LotArea.

The first model we will tackle is the simple pooled model. This is just taking all of our data regardless of neighborhood and creating a linear regression of the form: $$logSalePrice= intercept + \beta_{i}*LotArea + \epsilon$$

In [ ]:
ames.groupby(['Neighborhood']).agg({'log_LotArea': 'mean', 'log_SalePrice': 'mean' }).sort_values(by='log_SalePrice', ascending=False)

## Pooled Model

Lets fit a simple linear model using sklearn to compare to the results we get from pymc3

In [ ]:
linreg = LinearRegression()
linreg.fit(ames[['log_LotArea']], ames['log_SalePrice'])

fig, ax = plt.subplots(figsize=(12, 8))

x = np.linspace(6, 13)
ax.scatter(ames['log_LotArea'], ames['log_SalePrice'], alpha=.1)
ax.plot(x, linreg.predict(x.reshape(-1, 1)))
ax.set_title(f'Intercept: {linreg.} Beta: {linreg.coef_[0]:.2f} ')

In [ ]:
with pm.Model() as model:
    # Intercept prior
    alpha = pm.Normal('Intercept', mu=0, sd=1)
    # Slope prior
    beta = pm.Normal('log_LotArea', mu=0, sd=1)

    # Model error prior
    sigma = pm.HalfCauchy('sigma', beta=1)

    # Linear model
    price = alpha + beta * ames['log_LotArea'].values

    # Data likelihood
    y_like = pm.Normal('y_like', mu=price, sd=sigma, observed=ames['log_SalePrice'].values)

    # Inference button (TM)!
    trace = pm.sample(draws=2000, progressbar=True, tune=1000)

In [ ]:
pm.traceplot(trace);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

for _ in range(50):
    sample = np.random.choice(trace)
    x = np.linspace(6, 13)
    y = sample['Intercept'] + sample['log_LotArea']*x

    ax.plot(x, y, alpha=.1, color='orange')
ax.scatter(ames['log_LotArea'], ames['log_SalePrice'], alpha=.1)
ax.plot(x, linreg.predict(x.reshape(-1, 1)))

## Individual Non-Pooled Models

For each neighborhood we take all observations and construct and independent regression

In [ ]:
neighborhoods = ames['Neighborhood'].unique()

In [ ]:
traces = {}

for nh in neighborhoods:
    print(nh)
    lot_area = ames.loc[ames['Neighborhood'] == nh, 'log_LotArea'].values
    observed = ames.loc[ames['Neighborhood'] == nh, 'log_SalePrice'].values
    
    with pm.Model() as individual_model:
        # Intercept prior
        alpha = pm.Normal('Intercept', mu=0, sd=5)
        
        # Slope prior
        beta = pm.Normal('log_LotArea', mu=0, sd=5)

        # Model error prior
        sigma = pm.HalfCauchy('sigma', beta=1)

        # Linear model
        price = alpha + beta * lot_area

        # Data likelihood
        y_like = pm.Normal('y_like', mu=price, sd=sigma, observed=observed)

        # Inference button (TM)!
        indiv_trace = pm.sample(draws=4000, progressbar=True, tune=1000)
        traces[nh] = indiv_trace

In [ ]:
pm.plot_trace(traces['ClearCr']);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
nh = 'ClearCr'

for _ in range(100):
    sample = np.random.choice(traces[nh])
    x = np.linspace(6, 13)
    y = sample['Intercept'] + sample['log_LotArea']*x

    ax.plot(x, y, alpha=.1, color='orange')

ax.scatter(ames.loc[ames['Neighborhood'] == nh, ['log_LotArea']], ames.loc[ames['Neighborhood'] == nh, ['log_SalePrice']], alpha=.5)
ax.plot()

In [ ]:
neighborhood_idx = ames['Neighborhood_code'].values

with pm.Model() as hierarchical_model:
    # Hyperpriors
    mu_a = pm.Normal('mu_alpha', mu=0., sd=1)
    sigma_a = pm.HalfCauchy('sigma_alpha', beta=1)
    mu_b = pm.Normal('mu_beta', mu=0., sd=1)
    sigma_b = pm.HalfCauchy('sigma_beta', beta=1)
    
    # Intercept for each county, distributed around group mean mu_a
    a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape=ames['Neighborhood'].nunique())
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape=ames['Neighborhood'].nunique())
    
    # Model error
    eps = pm.HalfCauchy('eps', beta=1)
    
    # Expected value
    price_est = a[neighborhood_idx] + b[neighborhood_idx] * ames['log_LotArea'].values
    
    # Data likelihood
    y_like = pm.Normal('y_like', mu=price_est, sd=eps, observed=ames['log_SalePrice'].values)
    
    hierarchical_trace = pm.sample(draws=4000, progressbar=True, tune=1000)

In [ ]:
hier_a = hierarchical_trace['alpha'].mean(axis=0)
hier_b = hierarchical_trace['beta'].mean(axis=0)
indv_a = [traces[c]['Intercept'].mean() for c in neighborhoods]
indv_b = [traces[c]['log_LotArea'].mean() for c in neighborhoods]
fig, ax = plt.subplots(figsize=(10, 10))

ax.scatter(indv_a,indv_b, s=26, alpha=0.4, label = 'non-hierarchical')
ax.scatter(hier_a,hier_b, c='red', s=26, alpha=0.4, label = 'hierarchical')

for i_a, i_b, h_a, h_b in zip(indv_a, indv_b, hier_a, hier_b):
    ax.arrow(i_a, i_b, h_a - i_a, h_b - i_b,
             fc="k", ec="k", length_includes_head=True, alpha=0.1, head_width=.02)
    
ax.set_xlabel('Intercept')
ax.set_ylabel('log_LotArea')
ax.set_title('Hierarchical vs. Non-hierarchical Bayes')

In [ ]:
pm.plot_trace(hierarchical_trace, compact=True);

In [ ]:
pm.forestplot(hierarchical_trace);